### Compatibility notes
This notebook was originally written in Julia 0.6.x. There have been some changes in both syntax and definition of base functions. The following adaptions have been made to this notebook to make it compatible with newer versions (Julia 1.x).

* `transpose()` $\rightarrow$ `permutedims()` for data manipulation, same goes for `adjoint()`
* `sum(x, 1)` $\rightarrow$ `sum(x, dims = 1)` for summing over a specific dimension of an array `x`
* `1.^2 + z` $\rightarrow$ `1^2 .+ z` if `z` is an array of sorts
* `linspace(start, stop, n)` is deprecated, use `range(start, stop, length = n)` instead
* `zeros()` used to take any array as its argument, and return a zeros array with the same dimensions. No longer! You have to specify the dimensions. So instead of `zeros(A)` do `zeros(size(A))` if you want a zero array with the same size as `A`.
* It's not allowed anymore to set a range in an array equal to a scalar as in `A[start : end] = b`. Instead use `A[start : end] = b * ones(length( A[start : end] ))` (there's probably a neater way to do this, but it works).
* Creating an unintialized array no longer works with `Array{T, N}(dims)`, but you need to use `Array{T, N}(undef, dims)` instead.

In [3]:
using ForwardDiff
using Plots, Interact
using BenchmarkTools
Plots.plotly()

HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-2268509124596282030\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-2945761532665665871\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-2268509124596282030\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

Plots.PlotlyBackend()

In [4]:
function pathVec(pointArray::Array{Float64,2})
    #assume pointArray is an 2d array 3 X N
    #assume the path vector associated with the end of the path is a zero vector.
    return hcat(pointArray[:,2:end]-pointArray[:,1:end-1],[0.0, 0.0, 0.0])
end


function normCubed(x)
    #calculates |x|^3
    return sqrt.(sum(x.^2, dims = 1)).^3
end

function crossArray(x,y)
    #assume input arrays are 2d array 3 X N
    #returns 3XN array
    #return hcat(x[2,:].*y[3,:]-y[2,:].*x[3,:], x[3,:].*y[1,:]-y[3,:].*x[1,:],x[1,:].*y[2,:] - y[1,:].*x[2,:])'
    xX = @view x[1,:]
    xY = @view x[2,:]
    xZ = @view x[3,:]
    yX = @view y[1,:]
    yY = @view y[2,:]
    yZ = @view y[3,:]
    return hcat(xY.*yZ-yY.*xZ, xZ.*yX-yZ.*xX,xX.*yY - yX.*xY)'
end


function biasField(probePoint,pathElement,dlvec)
    #computes dl⃗ × r⃗./normCubed(rvec)³
    rvec = pathElement.-probePoint
    return sum(crossArray(dlvec,rvec)./normCubed(rvec), dims = 2)
end

function fieldNorm(x,pathElement,dlvec) 
   return sqrt(sum(biasField(x,pathElement,dlvec).^2))
end
#fieldNormFixed(x) = fieldNorm(x,path,dl)
#normGrad(x) = ForwardDiff.gradient(fieldNormFixed,x)
#normCurve(x) = ForwardDiff.hessian(fieldNormFixed,x)

fieldNorm (generic function with 1 method)

In [5]:
normGrad(x) = ForwardDiff.gradient(x->fieldNorm(x,path,dl),x)
normCurve(x) = ForwardDiff.hessian(x->fieldNorm(x,path,dl),x)

normCurve (generic function with 1 method)

In [6]:
#warmup - let JIT compiler compile the functions
angRes = 0.1;
t = 0.0:angRes:(2*pi*1.0);
path = permutedims(1.0.*hcat(cos.(t),sin.(t),zero(t)))
probeLocation = [0.0, 0.0, 0.0];
dl = pathVec(path);

fieldNorm(probeLocation,path,dl)
normGrad(probeLocation)
normCurve(probeLocation)

3×3 Array{Float64,2}:
 8.91913    0.0803761    0.0   
 0.0803761  9.64988      0.0   
 0.0        0.0        -18.5679

In [ ]:
wow = Array{Float64,1}();
for i=0.001:0.01:2
    push!(wow,normGrad([0.0,0.0,i])[3])
end

z = collect(0.001:0.01:2);
realGradZ = 2*pi*1*(1)*(-3*z).*(1^2 .+ z.^2).^(-2.5)
plot(z,wow,linewidth=2,title="JuliaDiff vs Analytical Grad")
plot!(z,realGradZ)

In [8]:
plt = plot(linewidth=2,title = "Numerical difference between JuliaDiff and Analytical Grad")
for angRes in [0.01,0.003,0.001,0.0001]
    t = 0.0:angRes:(2*pi*1.0);
    path = permutedims(1.0.*hcat(cos.(t),sin.(t),zero(t)))
    dl = pathVec(path);

    wow = Array{Float64,1}();
    for i=0.001:0.01:2
        push!(wow,normGrad([0.0,0.0,i])[3])
    end
    plot!(z,log10.(abs.((wow-realGradZ)./realGradZ)),label = "angRes = $angRes")
end

In [9]:
plt

<!DOCTYPE html>
 
 
 Plots.jl

In [10]:
bzDD = @. 2*pi*(1.0)*(-3.0)*(1.0^2-4.0*z^2)/sqrt((1.0^2 + z^2)^7)
wowww = Array{Float64,1}()
    for i in z
        push!(wowww,normCurve([0.0,0.0,i])[3,3])
    end
plot(z,wowww,linewidth=2,title="ForwardDiff.hessian vs Analytical Curvature")
plot!(z,bzDD)

<!DOCTYPE html>
 
 
 Plots.jl

### Define path and dl for the coil geometry being tested

assume we use 3.175 mm (1/16 '') outer side length hollow core wire.   
in computer simulation for Zeeman slower, Erik Streed assumes a physical length of 3.5 mm for the wire.    
I assume that takes finite thickness of epoxy layer into account?   
so, physical dimension of wire in calculation is 3.5 mm   

### description of the winding-counter-winding pair geometry

Starting from the positive lead, assume you wind spiraling in, clockwise, with N turns.   
After finishing N clockwise turns, you move in N x (wire thickness) radially.   
Then you change layer. Assume you go down.   
In the new layer, the helicity of the winding stays the same with respect to the z-axis, (clockwise) but you spiral OUT.
But the top layer does not completely overlap the bottom layer. There is some displacement of a wire thickness.

In [ ]:
#Let's try to describe the geometry with (5 spiral-,5 counterspiral-turns) with a 3D plot,
# note: n turns means total of n+1 thickness along the radial direction
function coilStack(startingAngle,turns,bucketWallRadius,wt=3.5,angRes=0.003,zpos = 0)
    ang = collect(range(startingAngle,2*pi*turns+startingAngle, length = Int(floor(2*pi*turns/angRes))))
    ang2 = collect(range(2*pi*turns+startingAngle,2*pi*2*turns+startingAngle, length = Int(floor(2*pi*turns/angRes))))
    # outer radius
    radius = bucketWallRadius + turns*wt+wt/2;
    coreTop = (radius.-ang/(2*pi) * wt).*hcat(cos.(ang), sin.(ang), zeros(size(ang)));
    coreTop[:,3] = zpos .+ fill(wt,length(ang),1)
    
    innerRadius = radius-ang[end]/(2*pi)*wt;
    transitionLength = Int(floor(0.2/angRes)); 
    coreBottom = (innerRadius.+(ang2.-ang[end])/(2*pi)*wt).*hcat(cos.(ang),sin.(ang), zeros(size(ang)))
    coreBottom[1:transitionLength,3] = ( zpos + wt ) .- wt/transitionLength *(1:1.0:transitionLength)
    coreBottom[transitionLength + 1 : end, 3] = zpos * ones( length(coreBottom[transitionLength + 1 : end, 3]) )
    return vcat(coreTop, coreBottom)
end
samplecoil = coilStack(0.0,5,28)
plt = path3d(samplecoil[:,1],samplecoil[:,2],samplecoil[:,3],zlim=(0,4),xlab = "x",ylab="y",zlab="z",color = "blue")

In [ ]:
#realistic coil stack
path = Array{Float64,2}(undef, 3,0)
dl = Array{Float64,2}(undef, 3,0)
plt2 = path3d()
#plt3 = path3d()
nstacks = 4;
for i=1:nstacks
    p = permutedims(coilStack(0.0+2*pi/nstacks * (i-1),4,28,3.5,0.002,-16.73-3.5/2 - 3.5*2*(i-1)))
    dp = pathVec(p)
    path = hcat(path,p)
    dl = hcat(dl,dp)
    path3d!(p[1,:],p[2,:],p[3,:],color = "red")
    #path3d!(dp[1,:],dp[2,:],dp[3,:])
end

#add bias coil stack
#2 turns,outer radius is 88.4 mm, inner radius is 77.8mm, maybe we can put up to 3 stacks

for i=1:3
    #bias coil on top
    pbT = permutedims(coilStack(0.0+2*pi/3*(i-1),2,77.8,3.5,0.002,32.08+3.5/2+3.5*2*(i-1)))
    pbB = permutedims(coilStack(0.0+2*pi/3*(i),2,77.8,3.5,0.002,-38.53-3.5/2-3.5*2*(i-1)))
    dpbT = -pathVec(pbT) # current runs in opposite direction
    dpbB = -pathVec(pbB)
    path = hcat(path,pbT,pbB)
    dl = hcat(dl,dpbT,dpbB)
    path3d!(pbT[1,:],pbT[2,:],pbT[3,:],color = "blue")
    path3d!(pbB[1,:],pbB[2,:],pbB[3,:],color = "blue")
end
plt2

In [62]:
bnorm = Array{Float64,1}();
bgrad = Array{Float64,1}();
bcx = Array{Float64,1}();
bcy = Array{Float64,1}();
bcz = Array{Float64,1}();
for i=-2:0.1:2
    push!(bnorm,fieldNorm([0.0,0.0,i],path,dl))
    push!(bgrad,normGrad([0.0,0.0,i])[3])
    nc = normCurve([0.0,0.0,i])
    push!(bcz,nc[3,3])
    push!(bcy,nc[2,2])
    push!(bcx,nc[1,1])
end

In [63]:
plot(-2:0.1:2,bgrad*10,title = "magnetic field gradient (G/(cm*Amp)")

<!DOCTYPE html>
 
 
 Plots.jl

In [64]:
plot(-2:0.1:2,bcx*100,title = "magnetic field curvature (G/(cm^2 * Amp))")
plot!(-2:0.1:2,bcy*100)
plot!(-2:0.1:2,bcz*100)

<!DOCTYPE html>
 
 
 Plots.jl

In [65]:
plot(-2:0.1:2,bnorm,title = "magnetic field norm (G/(Amp))")

<!DOCTYPE html>
 
 
 Plots.jl

In [67]:
@btime normCurve([0.0,0.0,0.0])

  390.629 ms (61 allocations: 493.93 MiB)


3×3 Array{Float64,2}:
  0.000604486  -2.57665e-6  -1.60088e-5
 -2.57665e-6    0.00060287   3.99879e-6
 -1.60088e-5    3.99879e-6   0.00243111

In [68]:
@btime normGrad([0.0,0.0,0.0])

  80.188 ms (56 allocations: 123.48 MiB)


3-element Array{Float64,1}:
  0.00011106464618383683
 -0.00012476917079490263
 -0.10873778760228053   

In [69]:
@btime fieldNorm([0.0,0.0,0.0],path,dl)

  18.926 ms (53 allocations: 30.87 MiB)


1.6248582816712764